<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/regression_targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install cityhash
import cityhash
from copy import deepcopy

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for cityhash: filename=cityhash-0.2.3.post9-cp36-cp36m-linux_x86_64.whl size=82879 sha256=cb4f157dc8fe3c1e0d43f0c50d584d65facccb0ef728be5b40e34f05ba556eb3
  Stored in directory: /root/.cache/pip/wheels/a4/4d/ae/cf0f1f8058ca2738fe972c69291083a0a06765271fb1047edd
Successfully built cityhash


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
df_names = [x[:-4] for x in file_names]
print(df_names)

dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x) for x in file_names]
df_dict = dict(zip(df_names, dfs))
print(df_dict.keys())

['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events']
dict_keys(['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events'])


In [4]:
df_dict["organizations"][:200]

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,123607.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,acquired,Wetpaint offers an online social publishing pl...,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,3.975000e+07,USD,2005-06-01,2008-05-19,NaN,51-100,info@wetpaint.com,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,8777.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Information Technology,Internet Services,Priva...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,11537.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,digg.com,http://www.digg.com,USA,NY,New York,New York,NaN,NaN,acquired,Digg Inc. operates a website that enables its ...,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,4.900000e+07,USD,2004-10-11,2016-09-13,NaN,51-100,feedback@digg.com,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,6844.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,omidyar.com,http://www.omidyar.com,USA,CA,California,Redwood City,1991 Broadway Suite 200,94063,operating,Omidyar Network is an investment firm.,"Enterprise Software,Financial Services,Venture...","Financial Services,Lending and Investments,Sof...",NaN,NaN,NaN,NaN,2004-01-01,NaN,NaN,101-250,info@omidyar.com,650.482.2500,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,investor,33.0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,15.0,2007-05-26 04:22:15,2020-03-30 18:33:35,"Facebook, Inc.","investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,ipo,Facebook is an online social networking servic...,"E-Commerce,Mobile Apps,Social,Social Media,Soc...","Apps,Commerce and Shopping,Content and Publish...",15.0,2.335700e+09,2.335700e+09,USD,2004-02-04,2013-06-30,NaN,10000+,NaN,NaN,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### Validation

In [5]:
print(np.shape(df_dict['organizations']))
print(np.shape(df_dict['funding_rounds']))

fr = df_dict['funding_rounds']['org_uuid'].apply(cityhash.CityHash64)
orgs = df_dict['organizations']['uuid'].apply(cityhash.CityHash64)
print(fr, orgs)

frogs = fr[fr.isin(orgs)]
print(frogs[0])
print(np.shape(frogs))

orgfrs = orgs[orgs.isin(fr)]
print(np.shape(orgfrs))

(1089733, 41)
(333176, 24)
0          5087506707876194815
1          5087506707876194815
2          5087506707876194815
3          1053843699762228743
4         17418110891307706341
                  ...         
333171     1693115704498827940
333172    11932824214984234785
333173    11932824214984234785
333174    11939097437904463371
333175     9204522222235410490
Name: org_uuid, Length: 333176, dtype: uint64 0          13685534557686295101
1            764015621929367586
2          10846552445983457719
3          10693046220981818130
4           5087506707876194815
                   ...         
1089728     7648380604111671063
1089729    15062640940509416191
1089730     9929788675484222851
1089731     2009917061565156091
1089732    12539503581336717831
Name: uuid, Length: 1089733, dtype: uint64
5087506707876194815
(333176,)
(177522,)


In [6]:
def maybe_add_funding_estimates(row):
  if np.isnan(row['post_money_valuation_usd']) and not np.isnan(row['raised_amount_usd']):
    if row['raised_amount_usd'] < 1:
      row['post_money_valuation_usd'] = 10000
    else:
      # print("Replaced: ", row['post_money_valuation_usd'], " ", row['raised_amount_usd']*3)
      row['post_money_valuation_usd'] = row['raised_amount_usd']*5
  return row['post_money_valuation_usd']

df = pd.DataFrame(list(zip([np.nan, np.nan], ['1000', '1000'])) ,columns =['post_money_valuation_usd', 'raised_amount_usd']).astype(np.float32)
df['post_money_valuation_usd'] = df.apply(maybe_add_funding_estimates, axis=1)
print(df)

   post_money_valuation_usd  raised_amount_usd
0                    5000.0             1000.0
1                    5000.0             1000.0


In [7]:
imputed_funding_rounds = deepcopy(df_dict["funding_rounds"])
# print(imputed_funding_rounds)
imputed_funding_rounds['post_money_valuation_usd'] = imputed_funding_rounds.apply(maybe_add_funding_estimates, axis=1)
# print(imputed_funding_rounds[['post_money_valuation_usd', 'raised_amount_usd']])

In [13]:
print(imputed_funding_rounds.head())

funding_data = imputed_funding_rounds[['org_uuid', 'announced_on', 'post_money_valuation_usd']].rename(
    columns={"announced_on": "date", "post_money_valuation_usd": "valuation"})

acquisitions_data = deepcopy(df_dict["acquisitions"][['acquiree_uuid', 'acquired_on', 'price_usd']]).rename(
    columns={"acquiree_uuid": "org_uuid", "acquired_on": "date", "price_usd": "valuation"})
ipos_data = deepcopy(df_dict["ipos"][['org_uuid', 'went_public_on', 'valuation_price_usd']]).rename(
    columns={"went_public_on": "date", "valuation_price_usd": "valuation"})

types = df_dict["acquisitions"][['type']].values
acquisition_types = df_dict["acquisitions"][['acquisition_type']].values
acquisitions_data['event'] = [types[x][0] if not isinstance(acquisition_types[x][0], str) 
                              else acquisition_types[x][0] for x in range(len(acquisitions_data))]
funding_data['event'] = pd.Series([x[0] for x in df_dict["funding_rounds"]['name'].str.split(" - ")])
ipos_data['event'] = df_dict['ipos']['stock_exchange_symbol']



                                   uuid  ...                                lead_investor_uuids
0  8a945939-18e0-cc9d-27b9-bf33817b2818  ...               3f47be49-2e32-8118-01a0-31685a4d0fd7
1  d950d7a5-79ff-fb93-ca87-13386b0e2feb  ...               b08efc27-da40-505a-6f9d-c9e14247bf36
2  6fae3958-a001-27c0-fb7e-666266aedd78  ...  e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-...
3  bcd5a63d-ed99-6963-0dd2-e36f6582f846  ...                                                NaN
4  60e6afd9-1215-465a-dd17-0ed600d4e29b  ...               fb2f8884-ec07-895a-48d7-d9a9d4d7175c

[5 rows x 24 columns]


In [14]:
imputed_funding_rounds.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,143236.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,500000.0,USD,2500000.0,NaN,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,145939.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,273250.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,California,Menlo Park,series_b,2006-04-01,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,149479.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,Colorado,Denver,series_b,2006-05-01,10500000.0,10500000.0,USD,52500000.0,NaN,USD,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,313010.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,California,West Hollywood,series_a,2007-01-17,NaN,NaN,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c


In [15]:
timelines = pd.concat([funding_data, acquisitions_data, ipos_data])

# Normalize dates

timelines.sort_values(by=['org_uuid', 'date'], inplace=True)
timelines.reset_index(inplace=True, drop=True)

timelines.set_index(['org_uuid'])
grouped_timelines = timelines.groupby(['org_uuid'])

In [16]:
grouped_timelines.head()
print(grouped_timelines['org_uuid'])

In [17]:
inspect_id = 'ffffabce-6d4a-b3d1-13c0-4e90cedf5270'
funding_data[funding_data['org_uuid'] == inspect_id].head()
# organizations[organizations['uuid'] == inspect_id].head()

,org_uuid,date,valuation,event
132839,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2015-05-01,400000.0,Angel Round
132840,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2015-01-01,75000.0,Grant
132841,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2015-09-15,410000.0,Product Crowdfunding
132842,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2016-05-15,1050000.0,Seed Round
150066,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2017-02-09,3500000.0,Seed Round


In [18]:
import datetime
company_timelines = {}

milli_per_day = 86400  # Used for converting Unix time
dropped = 0

num_timelines = len(timelines)
print("Processing ", num_timelines, " timelines")
iter = 0
for company_id in timelines['org_uuid']:
  if iter % 100000 == 0:
    print("Finished ", iter, "of ", num_timelines)
  iter += 1
  if company_id in company_timelines:
    # Already did this one. Move on
    continue
  timeline = grouped_timelines.get_group(company_id)
  timeline = timeline[timeline['valuation'].notnull()]               # We only care about times at which there is a valuation of the company.
  if timeline.empty:
    dropped += 1
    continue
  
  timeline = timeline.copy()
  timeline.reset_index(inplace=True, drop=True)

  day_zero = datetime.datetime.strptime(timeline['date'].iloc[0], '%Y-%m-%d').timestamp()
  parsed_dates = pd.Series([datetime.datetime.strptime(x, '%Y-%m-%d').timestamp() for x in timeline['date']])
  norm_dates = (parsed_dates - day_zero)/milli_per_day
  timeline['norm_dates'] = norm_dates
  company_timelines[company_id] = timeline

print("Retained: ", len(company_timelines))
print("Dropped: ", dropped)

Retained:  144569
Dropped:  152322


In [23]:
iter = 0
for x, y in company_timelines.items():
  print(x)
  iter += 1
  if iter > 5:
    break

00000aa4-ba42-9b68-a9c3-040c9f3bf9b9
00002470-bff7-6226-5800-0ca1b3787b6f
000095de-8e2b-82f1-32a7-c222ba3d5682
0000d497-c93a-eea3-eeb0-a943dfb4f71e
0001a8cc-0cdc-4a30-b4d3-da1b425069e1
0001eae7-077d-4d0b-a717-f67bcf2a09fa


### Validation

In [20]:
check_val = '00002470-bff7-6226-5800-0ca1b3787b6f'
print(company_timelines[check_val])
fr_data = df_dict['funding_rounds']
view = fr_data[fr_data['org_uuid'] == check_val]
print(np.shape(view))
hash_val = cityhash.CityHash64(check_val)
print("Hash Value: ", hash_val)
orgfrs_view = orgfrs[orgfrs == hash_val]
print(orgfrs_view)
print(orgfrs.eq(hash_val).any())

                               org_uuid        date  ...       event norm_dates
0  00002470-bff7-6226-5800-0ca1b3787b6f  2014-03-24  ...  Seed Round        0.0
1  00002470-bff7-6226-5800-0ca1b3787b6f  2015-08-26  ...  Seed Round      520.0
2  00002470-bff7-6226-5800-0ca1b3787b6f  2016-12-14  ...  Seed Round      996.0

[3 rows x 5 columns]
(5, 24)
Hash Value:  13360469805707984821
130002    13360469805707984821
Name: uuid, dtype: uint64
True


In [21]:
for company_id, timeline in company_timelines.items():
  initial_valuation = timeline['valuation'].iloc[0]
  timeline['initial_val'] = initial_valuation.tolist()
  norm_valuations = pd.Series([np.log(value/initial_valuation) for value in timeline['valuation']])
  timeline['norm_valuations'] = norm_valuations

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [25]:
from scipy import interpolate
from google.colab import files

def get_valuation_interpolations(timelines, day):
  company_hashes = []
  regression_targets = []
  initial_valuations = []
  for company, timeline in timelines.items():
    x = timeline['norm_dates'].tolist()
    y = timeline['norm_valuations'].tolist()

    hash = cityhash.CityHash64(company)
    company_hashes.append(hash)
    initial_valuations.append(timeline['initial_val'].tolist()[0])

    if len(x) < 2 or len(y) < 2:
      regression_targets.append(0)     # Single point timeline. Leave decision to drop for downstream.
      continue
    
    if day > x[-1]:
      regression_targets.append(y[-1])  # Don't extrapolate the data at all.
      continue

    interpolator = interpolate.interp1d(x, y)
    regression_target = interpolator(day)
    # print(company)
    # print(frogs)
    if not orgfrs.eq(hash_val).any():
      print("Bad Hash")
      break
    regression_targets.append(regression_target)

  return pd.DataFrame(list(zip(company_hashes, initial_valuations, regression_targets)), columns=["hash", "initial_valuation", "log_valuation_factor"])


In [27]:
for day in [200, 500, 1000, 2000]:
  frame = get_valuation_interpolations(company_timelines, day)
  print(frame)
  print(np.count_nonzero(frame['log_valuation_factor']))
  frame.to_csv('/content/gdrive/My Drive/vc_modeling/regression_targets/' + str(day) + '.csv')


                        hash  initial_valuation log_valuation_factor
0        2053339725337568679        413036820.0                    0
1       13360469805707984821          3000000.0   0.2665950694461328
2       12201126308526847683         47500000.0                    0
3       17482404514494389050          2157880.0                    0
4       16923506324318240851          7500000.0                    0
...                      ...                ...                  ...
144564   4057326795460754576           552105.0                    0
144565  17393885764651115266         20000000.0    1.011831456598829
144566  14785394360939257924        125000000.0                    0
144567  15207057269115911424         31150000.0                    0
144568  12723450708549610702            75000.0   1.6883954774200638

[144569 rows x 3 columns]
53615
                        hash  initial_valuation log_valuation_factor
0        2053339725337568679        413036820.0                    0
1